In [1]:
%install-location $cwd/swift-install
%install '.package(url: "https://github.com/mxcl/Path.swift", from: "0.16.1")' Path
%install '.package(url: "https://github.com/saeta/Just", from: "0.7.2")' Just
%install '.package(url: "https://github.com/latenitesoft/NotebookExport", from: "0.5.0")' NotebookExport

Installing packages:
	.package(url: "https://github.com/mxcl/Path.swift", from: "0.16.1")
		Path
	.package(url: "https://github.com/saeta/Just", from: "0.7.2")
		Just
	.package(url: "https://github.com/latenitesoft/NotebookExport", from: "0.5.0")
		NotebookExport
With SwiftPM flags: []
Working in: /tmp/tmpgeqz_a9j/swift-install
[1/3] Compiling jupyterInstalledPackages jupyterInstalledPackages.swift
[2/4] Merging module jupyterInstalledPackages
[3/3] Linking libjupyterInstalledPackages.so
Initializing Swift...
Installation complete!


In this notebook, we implement from scratch an MPC (Pond) protocol. The Swift implementation is based on the implementation in [TF Encrypted](https://github.com/tf-encrypted/tf-encrypted) and from [Motern Dahl's github repository](https://github.com/mortendahl/privateml/blob/master/image-analysis/pond/tensor.py). Note that the goal is just to experiment with Swift TensorFlow. There are lot of missing components to make it fully secure. 

To learn more about encrypted machine learning with MPC, you can learn [this blog](https://mortendahl.github.io/2017/04/17/private-deep-learning-with-mpc/).

### Fixed-point Encoding

In [2]:
// export

// Source: The original python implementation can be found here:
// https://github.com/mortendahl/privateml/blob/master/image-analysis/pond/tensor.py

import TensorFlow

let SCALING_BASE = 3
let PRECISION_INTEGRAL   = 7
let PRECISION_FRACTIONAL = 8
let MATMUL_THRESHOLD = 256
let TRUNCATION_GAP = 20

let SCALING_FACTOR = pow(Float(SCALING_BASE), Float(PRECISION_FRACTIONAL))
let TOTAL_PRECISION = PRECISION_INTEGRAL + PRECISION_FRACTIONAL
let BOUND_SINGLE_PRECISION = Int64(pow(Float(SCALING_BASE), Float(TOTAL_PRECISION)))
let NATIVE_MIN = Int64.min
let NATIVE_MAX = Int64.max


extension TensorShape {
  func countElements() -> Int{
    var nb_el = 1
    for i in 0 ..< self.count{
      nb_el = nb_el * self[i]
    }
    return nb_el
  }
}

func _encode(rationals: Tensor<Float>) -> Tensor<Int64>{
    let scaled = rationals * SCALING_FACTOR 
    let integer = Tensor<Int64>(scaled)
    return integer
}

func _decode(field_element: Tensor<Int64>) -> Tensor<Float>{
    let bound = BOUND_SINGLE_PRECISION
    let scaled = (field_element + bound) - bound
    let scaled_rationals = Tensor<Float>(scaled)
    return scaled_rationals / Float(SCALING_FACTOR)
}

// NOTE Should be able to use Tensor(randomUniform instead)
func makeRandomList(_ n: Int, min: Int64 = NATIVE_MIN, max: Int64 = NATIVE_MAX) -> [Int64] {
    return (0..<n).map { _ in Int64.random(in: min..<max) }
}

func _share(x: Tensor<Int64>) -> (Tensor<Int64>, Tensor<Int64>){
    let share0 = Tensor<Int64>(makeRandomList(x.shape.countElements())).reshaped(to: x.shape)
    let share1 = x - share0
    return (share0, share1)
}

func reconstruct(share0: Tensor<Int64>, share1: Tensor<Int64>) -> Tensor<Int64>{
  let rec = share0 + share1
  return rec
}

In [3]:
public func testNearZero(_ a: Tensor<Float>, _ tolerance: Float) {
    assert(Raw.less(Raw.abs(a), Tensor<Float>([tolerance])).all())
}

public func testAlmostEqual(_ a: Tensor<Float>, 
                            _ b: Tensor<Float>, 
                            _ tolerance: Float = 1e-1) {
    assert(a.shape == b.shape)
    testNearZero(a-b, tolerance)
}

let x: Tensor<Float> = Tensor([[1.0, 2.0], [3.0, 4.0]])
let x_enc = _encode(rationals: x)
let x_shared = _share(x: x_enc)
let x_rec = _decode(field_element: reconstruct(share0: x_shared.0, share1: x_shared.1))

testAlmostEqual(x, x_rec)

### Encrypted Tensor

In [4]:
// export

// PublicTensor

public struct PublicTensor {
  let elements: Tensor<Int64>

  init(elements: Tensor<Int64>){
    self.elements = elements
  }

  public static func from_values(values: Tensor<Float>) -> PublicTensor{
    let enc_val = _encode(rationals: values)

    return PublicTensor(elements: enc_val)
  }

  public func unwrap() -> Tensor<Int64>{
    return self.elements
  }

  public func reveal() -> PublicTensor{
    return self
  }

  public func decode() -> Tensor<Float>{
    return _decode(field_element: self.elements)
  }
    
  public func truncate() -> PublicTensor {
    let positive_numbers = Tensor<Int64>(_Raw.lessEqual(self.elements, Tensor<Int64>([0])))
    var elements = self.elements
    elements = (2 * positive_numbers - 1) * elements
    elements = _Raw.floorDiv(elements, Tensor<Int64>([Int64(SCALING_FACTOR)]))
    elements = (2 * positive_numbers - 1) * elements

    return PublicTensor(elements: elements)
   }
 }


func add(x: PublicTensor, y: PublicTensor) -> PublicTensor{
    let elements = x.elements + y.elements
    return PublicTensor(elements: elements)
}
    
func mul(x: PublicTensor, y: PublicTensor) -> PublicTensor{
    let elements = _Raw.mul(x.elements, y.elements)
    return PublicTensor(elements: elements).truncate()
}


public func matmul(x: PublicTensor, y: PublicTensor) -> PublicTensor{
    let elements = _Raw.matMul(x.elements, y.elements)
    return PublicTensor(elements: elements).truncate()
}


// PublicFieldTensor

public struct PublicFieldTensor {
  let elements: Tensor<Int64>

  init(elements: Tensor<Int64>){
    self.elements = elements
  }

  public static func from_values(values: Tensor<Int64>) -> PublicFieldTensor{
    let enc_val = values
    return PublicFieldTensor(elements: enc_val)
  }

  public func reveal() -> PublicFieldTensor {
    return self
  }
}

public func add(x: PublicFieldTensor, y: PublicFieldTensor) -> PublicFieldTensor{
    let elements = x.elements + y.elements
    return PublicFieldTensor(elements: elements)
  }

public func add(x: PublicFieldTensor, y: PrivateFieldTensor) -> PrivateFieldTensor{
    let share0 = x.elements + y.share0
    let share1 = y.share1
    return PrivateFieldTensor(share0: share0, share1: share1)
}

func matmul(x: PublicFieldTensor, y: PublicFieldTensor) -> PublicFieldTensor{
    let elements = Raw.matMul(x.elements, y.elements)
    return PublicFieldTensor(elements: elements)
}

func matmul(x: PublicFieldTensor, y: PrivateFieldTensor) -> PrivateFieldTensor{
    let share0 = Raw.matMul(x.elements, y.share0)
    let share1 = Raw.matMul(x.elements, y.share1)
    return PrivateFieldTensor(share0: share0, share1: share1)
}


// PrivateFielTensor 

public struct PrivateFieldTensor {
  let share0: Tensor<Int64>
  let share1: Tensor<Int64>

  init(share0: Tensor<Int64>, share1: Tensor<Int64>){
    self.share0 = share0
    self.share1 = share1
  }

  public static func from_values(values: Tensor<Int64>) -> PrivateFieldTensor{
    let x_shared = _share(x: values)
    return PrivateFieldTensor(share0: x_shared.0, share1: x_shared.1)
  }

  public func reveal() -> PublicFieldTensor{
  let x_rec = reconstruct(share0: self.share0, share1: self.share1)
    return PublicFieldTensor.from_values(values: x_rec)
  }
}

public func add(x: PrivateFieldTensor, y: PrivateFieldTensor) -> PrivateFieldTensor{
    let share0 = x.share0 + y.share0
    let share1 = x.share1 + y.share1
    return PrivateFieldTensor(share0: share0, share1: share1)
}
    
 public func add(x: PrivateFieldTensor, y: PublicFieldTensor) -> PrivateFieldTensor{
    let share0 = x.share0 + y.elements
    let share1 = x.share1
    return PrivateFieldTensor(share0: share0, share1: share1)
  }

public func matmul(x: PrivateFieldTensor, y: PublicFieldTensor) -> PrivateFieldTensor{
    let share0 = Raw.matMul(x.share0, y.elements) 
    let share1 = Raw.matMul(x.share1, y.elements)
    return PrivateFieldTensor(share0: share0, share1: share1)
    }

In [5]:
let x_p = PublicTensor.from_values(values: Tensor<Float>([[1, 2], [3, 4]]))
let y_p = PublicTensor.from_values(values: Tensor<Float>([[1, 0], [0, 1]]))
let out_p = add(x:x_p , y:y_p)

testAlmostEqual(out_p.reveal().decode(), Tensor<Float>([[2.0, 2.0], [3.0, 5.0]]))

In [6]:
// export

public struct PrivateTensor {
  let share0: Tensor<Int64>
  let share1: Tensor<Int64>

  var shape: TensorShape {
        return share0.shape
    }

  init(share0: Tensor<Int64>, share1: Tensor<Int64>){
    self.share0 = share0
    self.share1 = share1
  }

  public static func from_values(values: Tensor<Float>) -> PrivateTensor{
    let x_shared = _share(x: _encode(rationals: values))
    return PrivateTensor(share0: x_shared.0, share1: x_shared.1)
  }

  public func reveal() -> PublicTensor{
    let x_rec = _decode(field_element: reconstruct(share0: self.share0, share1: self.share1))
    return PublicTensor.from_values(values: x_rec)
  }

  public func truncate() -> PrivateTensor {
    let share0 = _Raw.floorDiv(self.share0, Tensor<Int64>([Int64(SCALING_FACTOR)]))
    let share1 =  _Raw.floorDiv(self.share1, Tensor<Int64>([Int64(SCALING_FACTOR)]))
    return PrivateTensor(share0: share0, share1: share1)
   }
}

func generate_dot_triple(m: Int, n: Int, o: Int) -> (PrivateFieldTensor, 
                                                     PrivateFieldTensor, 
                                                     PrivateFieldTensor){

    let a = Tensor<Int64>(makeRandomList(m*n)).reshaped(to: [m, n])
    let b = Tensor<Int64>(makeRandomList(n*o)).reshaped(to: [n, o])
    let ab = Raw.matMul(a, b)
    return (PrivateFieldTensor.from_values(values: Tensor<Int64>(a)), 
            PrivateFieldTensor.from_values(values: Tensor<Int64>(b)), 
            PrivateFieldTensor.from_values(values: Tensor<Int64>(ab)))
}

public func add(x: PrivateTensor, y: PublicTensor) -> PrivateTensor{
    let share0 =  x.share0 + y.elements
    let share1 = x.share1
    return PrivateTensor(share0: share0, share1: share1)
  }
    
public func add(x: PrivateTensor, y: PrivateTensor) -> PrivateTensor{
    let share0 = x.share0 + y.share0
    let share1 = x.share1 + y.share1
    return PrivateTensor(share0: share0, share1: share1)
}

public func + (x: PrivateTensor, y: PrivateTensor) -> PrivateTensor{
    return add(x: x, y: y)
}

public func sub(x: PrivateTensor, y: PublicTensor) -> PrivateTensor{
    let share0 = x.share0 - y.elements
    let share1 = x.share1
    return PrivateTensor(share0: share0, share1: share1)
}

public func sub(x: PrivateTensor, y: PrivateTensor) -> PrivateTensor{
    let share0 = x.share0 - y.share0
    let share1 = x.share1 - y.share1
    return PrivateTensor(share0: share0, share1: share1)
}

public func sub(x: PrivateTensor, y: PrivateFieldTensor) -> PrivateFieldTensor{
  let share0 = x.share0 - y.share0
  let share1 = x.share1 - y.share1
  return PrivateFieldTensor(share0: share0, share1: share1)
}


public func matmul(x: PrivateTensor, y: PrivateTensor) -> PrivateTensor{
    let m = x.shape[0]
    let n = x.shape[1]
    let o = y.shape[1]

    let triple = generate_dot_triple(m: m, n: n, o: o)
    let a = triple.0 
    let b = triple.1 
    let ab = triple.2

    let alpha_r = sub(x: x, y: a).reveal()
    let beta_r = sub(x: y, y: b).reveal()
    let z_1 = matmul(x: alpha_r, y: beta_r)
    let z_2 = matmul(x: alpha_r, y: b)
    let z_3 = matmul(x: a, y: beta_r)
    let z = add(x: add(x: add(x: z_1, y: z_2), y: z_3), y: ab)

    return PrivateTensor(share0: z.share0 , share1: z.share1).truncate()
}

In [7]:
let a = PrivateTensor.from_values(values: Tensor([[1, 2], [3, 4]]))
let b = PrivateTensor.from_values(values: Tensor([[1, 2], [3, 4]]))
let c = PrivateTensor.from_values(values: Tensor([[1, 0], [0, 1]]))
let res_add = a + b
let res_sub = sub(x: a, y: b)
let res_matmul = matmul(x: a, y: c)

testAlmostEqual(res_add.reveal().decode(), Tensor<Float>([[2.0, 4.0], [6.0, 8.0]]))
testAlmostEqual(res_sub.reveal().decode(), Tensor<Float>([[0.0, 0.0], [0.0, 0.0]]))
testAlmostEqual(res_matmul.reveal().decode(), Tensor<Float>([[1.0, 2.0], [3.0, 4.0]]))

In [8]:
import NotebookExport
import Path
let exporter = NotebookExport(Path.cwd/"encrypted_tensor.ipynb")
print(exporter.export(usingPrefix: "ppmlNB_"))

success
